## network construction
There are three parts of the network. 
- PPI
- drug target interactions
- AD gene interactions

Combine the three parts to construct a network with AD, drug, gene as nodes and interactions as edges.

### 1, input the PPI network, drug_target info and the gene_ID matching file, AD gene info

In [1]:
import pandas as pd
import os

os.chdir('/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/')

raw_PPI_net = pd.read_csv("/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/raw_data/PPI/PPI.csv")
raw_PPI_net

,proteinA_entrezid,proteinB_entrezid,databases
0,9796,56992,bioplex|HIPPIE
1,7918,9240,instruct|hiunion|APID|biogrid|HINT-binary|intact
2,8233,23548,bioplex|HIPPIE
3,4899,11253,encode
4,5297,8601,bioplex
...,...,...,...
327919,116135,286451,intact|hiunion
327920,1105,8451,qubic
327921,1655,7307,CoFrac
327922,4089,55363,mint|APID|intact


In [2]:
ori_drug_target_inter = pd.read_csv("/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/Drug-target/Drug_target_JackList/new.filtered_Final_drug_target_info_2.txt", sep = "\t")
drug_list = pd.read_csv("/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/Drug-target/Drug_target_JackList/filteredCountsPostExclusion.csv",sep=",")
drugName = drug_list['type']
drug_target_inter = ori_drug_target_inter[ori_drug_target_inter['Drug_name'].isin(drugName)]
drug_target_inter 

,Drug_name,DrugBank_ID,CHEMBL_ID,Gene_name,UniProt,Source,EntrezID
38,acarbose,DB00284,CHEMBL1566,AMY1A,P0DUB6,CHEMBL,276
39,acarbose,DB00284,CHEMBL1566,GAA,P10253,CHEMBL,2548
40,acebutolol,DB01193,CHEMBL642,ADRB1,P08588,"CHEMBL,IUPHAR,BindingDB",153
41,acebutolol,DB01193,CHEMBL642,ADRB2,P07550,BindingDB,154
44,acetaminophen,DB00316,CHEMBL112,CA1,P00915,"CHEMBL,TTD",759
...,...,...,...,...,...,...,...
9957,megestrol,DB00351,CHEMBL1201139,NR3C1,P04150,CHEMBL,2908
9958,mometasone,DB14512,CHEMBL1161,NR3C1,P04150,CHEMBL,2908
9959,mometasone,DB14512,CHEMBL1161,PGR,P06401,CHEMBL,5241
9961,sodium,DB09154,CHEMBL1355,CA4,P22748,CHEMBL,762


In [3]:
len(set(drugName))

387

In [4]:
set(drugName).difference(set(drug_target_inter['Drug_name']))

{'metformin'}

In [3]:
AD_gene_inter = pd.read_csv("/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/AD_gene/filtered_all_AD_genes.txt",sep="\t")
AD_gene_inter

,Gene,EntrezID
0,SHARPIN,81858
1,UMAD1,729852
2,CR1,1378
3,CDK5,1020
4,HS3ST5,222537
...,...,...
225,DOC2A,8448
226,TMEM106B,54664
227,ARC,23237
228,ADAM17,6868


In [4]:
geneIDs = raw_PPI_net['proteinA_entrezid'].append(raw_PPI_net['proteinB_entrezid']).append(drug_target_inter['EntrezID']).append(AD_gene_inter['EntrezID'])
max(geneIDs)

114483834

### the range of gene Entrez ID is 1-114483834, so assign the AD node ID as 114483835. There are 386 unique drugs

In [5]:
114483835+386

114484221

### 2, use the entrze ID of gene as gene node ID, use 114483835 as AD node ID, use 114483836-114484221 as drug node ID

In [7]:
drugName_nodeID = {}
drugName_nodeID['Drug_name'] = drug_target_inter['Drug_name'].unique()
drugName_nodeID['NodeID'] = [i+114483836 for i in range(386)]
drugName_nodeID_df = pd.DataFrame.from_dict(drugName_nodeID)
drugName_nodeID_df.columns = ['Drug_name','NodeID']
drugName_nodeID_df.to_csv("/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/Drug-target/Drug_target_JackList/all_drugName_nodeID_with386drugs.txt",sep="\t",index=None)
drug_target_net = pd.merge(drug_target_inter,drugName_nodeID_df,how='left',on='Drug_name')
drug_target_net.to_csv("/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/Drug-target/Drug_target_JackList/Final_drug_target_info_withNodeID_with386drugs.txt", sep = "\t")
drug_target_net=drug_target_net[['EntrezID','NodeID','Source']]
drug_target_net.columns = ['Node1','Node2','Source']
drug_target_net.to_csv("/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/Graphs_and_embeddings/Graph_JackList/drug_target_net_with386drugs.txt",index=None,sep="\t")

- split drug nodes into 39 parts, with each part about 10 drugs

In [8]:
for i in range(39):
    start = i*10
    end=10*(i+1)
    if end>=386:
        end = 386
    file = "/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/Drug-target/Drug_target_JackList/Drug_list_to_run/all_drugName_nodeID_with386drugs_" + str(i) + ".txt"
    indivi_drugName_nodeID = pd.DataFrame()
    indivi_drugName_nodeID = drugName_nodeID_df.iloc[start:end,:]
    indivi_drugName_nodeID.to_csv(file,sep="\t",index=None)

In [11]:
AD_gene_inter['ADnode'] = [114483835] * AD_gene_inter.shape[0]
AD_gene_net = AD_gene_inter.iloc[:,1:3]
AD_gene_net.columns = ['Node1','Node2']
AD_gene_net['Source'] = ['ADnode']*AD_gene_net.shape[0]
AD_gene_net.to_csv("/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/Graphs_and_embeddings/New_graph/AD_gene_net.txt",index=None,sep="\t")
AD_gene_net

,Node1,Node2,Source
0,81858,114483835,ADnode
1,729852,114483835,ADnode
2,1378,114483835,ADnode
3,1020,114483835,ADnode
4,222537,114483835,ADnode
...,...,...,...
225,8448,114483835,ADnode
226,54664,114483835,ADnode
227,23237,114483835,ADnode
228,6868,114483835,ADnode


In [12]:
PPI_net = raw_PPI_net
PPI_net.columns = ['Node1','Node2','Source']
PPI_net.to_csv("/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/Graphs_and_embeddings/New_graph/PPI_net.txt",index=None,sep="\t")
PPI_net

,Node1,Node2,Source
0,9796,56992,bioplex|HIPPIE
1,7918,9240,instruct|hiunion|APID|biogrid|HINT-binary|intact
2,8233,23548,bioplex|HIPPIE
3,4899,11253,encode
4,5297,8601,bioplex
...,...,...,...
327919,116135,286451,intact|hiunion
327920,1105,8451,qubic
327921,1655,7307,CoFrac
327922,4089,55363,mint|APID|intact


In [15]:
all_gene_nodes = drug_target_net['Node1'].append(AD_gene_net['Node1']).append(PPI_net['Node1']).append(PPI_net['Node2'])
len(all_gene_nodes.unique())

18527

In [16]:
uniq_all_gene_nodes = all_gene_nodes.unique()
from numpy import savetxt
savetxt("/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/Drug-target/Drug_target_JackList/uniq_all_gene_nodes_with386drugs.txt", uniq_all_gene_nodes, fmt="%i",delimiter='\n')
len(uniq_all_gene_nodes)

18527

## R code to get the matching gene symbol and entrezid
```
library(org.Hs.eg.db)

setwd("C:/Drug_repurposing_project/Data")
uniq_all_gene_nodes=read.delim("uniq_all_gene_nodes.txt",stringsAsFactors = F,header = F)
uniq_geneSymbol <- bitr(as.character(uniq_all_gene_nodes$V1), fromType = "ENTREZID",
                        toType = c("SYMBOL"),
                        OrgDb = org.Hs.eg.db)
dim(uniq_geneSymbol)
write.table(uniq_geneSymbol,file = "uniq_all_gene_nodes_IDSymbol.txt",quote = F,row.names = F,col.names = F)

```


### generate the node ID-name list and store for query

In [17]:
uniq_all_gene_nodes_IDSymbol = pd.read_csv("/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/Drug-target/Drug_target_JackList/uniq_all_gene_nodes_IDSymbol_with386drugs.txt",sep=" ", index_col=None)
uniq_all_gene_nodes_IDSymbol.columns = ['NodeID','Name']
uniq_all_gene_nodes_IDSymbol
#uniq_all_gene_nodes_IDSymbol.loc[uniq_all_gene_nodes_IDSymbol["NodeID"].isin(uniq_all_gene_nodes)]

,NodeID,Name
0,2548,GAA
1,153,ADRB1
2,154,ADRB2
3,759,CA1
4,771,CA12
...,...,...
18511,26254,OPTC
18512,348158,ACSM2B
18513,221391,OPN5
18514,50651,SLC45A1


In [18]:
uniq_all_gene_nodes_IDSymbol.loc[uniq_all_gene_nodes_IDSymbol["NodeID"].isin(uniq_all_gene_nodes)]
drug_IDname = drugName_nodeID_df[['NodeID','Drug_name']]
drug_IDname.columns = ['NodeID','Name']
Final_NodeID_Name = drug_IDname.append(uniq_all_gene_nodes_IDSymbol)
Final_NodeID_Name.loc[len(Final_NodeID_Name.index)]=[114483835,'AD']
Final_NodeID_Name

,NodeID,Name
0,114483836,acarbose
1,114483837,acebutolol
2,114483838,acetaminophen
3,114483839,acetazolamide
4,114483840,aclidinium
...,...,...
18512,348158,ACSM2B
18513,221391,OPN5
18514,50651,SLC45A1
18515,201625,DNAH12


In [19]:
Final_NodeID_Name.to_csv("/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/Drug-target/Drug_target_JackList/Final_NodeID_Name_with386drugs.txt",index=None,sep="\t")

### 3, combine the three part to one dataframe and output it to file

In [20]:
Final_net = PPI_net.append(drug_target_net).append(AD_gene_net)
## add weight 1
Final_net['weight']=[1]*Final_net.shape[0]
Final_net = Final_net[['Node1','Node2','weight','Source']]
Final_net

,Node1,Node2,weight,Source
0,9796,56992,1,bioplex|HIPPIE
1,7918,9240,1,instruct|hiunion|APID|biogrid|HINT-binary|intact
2,8233,23548,1,bioplex|HIPPIE
3,4899,11253,1,encode
4,5297,8601,1,bioplex
...,...,...,...,...
225,8448,114483835,1,ADnode
226,54664,114483835,1,ADnode
227,23237,114483835,1,ADnode
228,6868,114483835,1,ADnode


In [21]:
Final_net.to_csv("/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/Drug-target/Drug_target_JackList/Final_network_with386drugs.txt",index=None,header=None,sep=" ")
Final_net_forNetworkx = Final_net[["Node1","Node2","weight"]]
Final_net_forNetworkx.to_csv("/fs/ess/PCON0041/shunian/AD_drugRep_project_202209/Data/Intermediate_data/Graphs_and_embeddings/Graph_JackList/Final_network_with386drugs_forDeepwalk.txt",index=None,header=None,sep=" ")

### Final, there are 336,311 interactions in the network, including 327,924 PPI interactions, 230 AD gene interactions, 8,157 drug target interactions, involving 920 drug nodes, 18,536 gene nodes, 1 AD node. In total there is 19,457 nodes in the network (for the 920 drug list)

### Final, there are 331,008 interactions in the network, including 327,924 PPI interactions, 230 AD gene interactions, 2,854 drug target interactions, involving 386 drug nodes, 18,527 gene nodes, 1 AD node. In total there is 18,914 nodes in the network (for the 386 drug list)

In [24]:
386+18527+1


18914